# Query store data with Spark SQL

### 1) Create a Spark session

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .appName("Physical Store Returns Processing")\
  .master("yarn")\
  .enableHiveSupport()\
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/15 13:02:34 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
24/07/15 13:02:34 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
24/07/15 13:02:34 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
24/07/15 13:02:34 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
# Display spark session properties
spark

### 2) Read the combined returns data from the Parquet file

In [3]:
# Retrieve the current project id and store in the variable called PROJECT_ID
PROJECT_ID=!gcloud info --format='value(config.project)'

# Read data from Parquet files in Google Cloud Storage and populate a DataFrame
# The Google Cloud Storage bucket name in this lab is the name of the project id
# We are using the project_id value obtained in the prior step in the gs:// path
store_returns_with_addr_dataframe = spark.read.parquet("gs://" + PROJECT_ID[0] + "/store_returns_output/store_returns.parquet/*.parquet")


In [4]:
# Print schema to review field names and data types
store_returns_with_addr_dataframe.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- rma_id: integer (nullable = true)
 |-- return_status: string (nullable = true)
 |-- status_date: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity_returned: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- street_address: string (nullable = true)



In [5]:
display(store_returns_with_addr_dataframe.show(3,truncate=False))

+--------+------+-------------+-----------------------+----------+-----------------+--------+---------------------------+
|order_id|rma_id|return_status|status_date            |product_id|quantity_returned|store_id|street_address             |
+--------+------+-------------+-----------------------+----------+-----------------+--------+---------------------------+
|44176   |39613 |in progress  |null                   |25059     |1                |20000   |000 Amber Viaduct Suite 499|
|16558   |1440  |complete     |2020-03-18 13:46:00 UTC|8875      |1                |20000   |000 Amber Viaduct Suite 499|
|29503   |34936 |complete     |null                   |3883      |1                |20000   |000 Amber Viaduct Suite 499|
+--------+------+-------------+-----------------------+----------+-----------------+--------+---------------------------+
only showing top 3 rows



None

In [6]:
# Create a view so that the DataFrame store_returns_with_address can be referenced as a table in Spark SQL
store_returns_with_addr_dataframe.createOrReplaceTempView("store_returns_with_address")

In [7]:
# Show store returns with address using Spark SQL
spark.sql("select * from store_returns_with_address").show(3, truncate=False)

+--------+------+-------------+-----------------------+----------+-----------------+--------+---------------------------+
|order_id|rma_id|return_status|status_date            |product_id|quantity_returned|store_id|street_address             |
+--------+------+-------------+-----------------------+----------+-----------------+--------+---------------------------+
|44176   |39613 |in progress  |null                   |25059     |1                |20000   |000 Amber Viaduct Suite 499|
|16558   |1440  |complete     |2020-03-18 13:46:00 UTC|8875      |1                |20000   |000 Amber Viaduct Suite 499|
|29503   |34936 |complete     |null                   |3883      |1                |20000   |000 Amber Viaduct Suite 499|
+--------+------+-------------+-----------------------+----------+-----------------+--------+---------------------------+
only showing top 3 rows



### 3) Returns by month using Spark SQL

In this section you will run a query that generates the same results you have seen in BigQuery before, but processed by Dataproc Spark and executed in a Jupyter Notebook.

The temporary view store_returns_with_address in the "from" line references a Spark DataFrame that contains data from the Parquet file in Google Cloud Storage. In BigQuery, the "from" line references a BigQuery standard table.

In [8]:
# Test the query you ran in BigQuery
dataframe = spark.sql('''
    select
    substring(status_date, 1, 7) as year_month,
    return_status,
    count(order_id) as order_count
    from store_returns_with_address
    group by year_month, return_status
''')
dataframe.show()

+----------+-------------+-----------+
|year_month|return_status|order_count|
+----------+-------------+-----------+
|   2020-07|      unknown|          6|
|   2021-05|     complete|         67|
|   2019-07|      unknown|          1|
|   2022-01|      unknown|         18|
|   2022-10|  in progress|         44|
|   2020-06|      unknown|          7|
|   2019-12|      unknown|          2|
|   2022-03|      unknown|         24|
|   2023-02|      unknown|         44|
|   2019-06|      unknown|          1|
|   2023-02|     complete|        197|
|   2021-04|      unknown|         15|
|   2023-03|      unknown|         50|
|   2019-05|     complete|          6|
|   2020-01|  in progress|         11|
|   2023-06|      unknown|         43|
|   2021-02|      unknown|         10|
|   2021-08|     complete|         76|
|   2019-09|  in progress|          3|
|   2022-10|      unknown|         40|
+----------+-------------+-----------+
only showing top 20 rows



### Return to the lab instructions